In [1]:
import prop
import pandas as pd
import matplotlib.pyplot as plt
from modules import pump, regen, heater, turbine, condenser

#Таблица с потоками и блоками:
streams = pd.read_excel("streams.xlsx",index_col=0)
blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

#Параметры нагревающей среды:
gas  = 'REFPROP::N2[0.7757]&O2[0.1068]&CO2[0.0193]&H2O[0.0936]&Ar[0.0046]' #Состав газа
Tin  = 183.6                                                               #Температура газов на входе
Pgas = 0.1                                                                 #Давление газов
Ggas = 509                                                                 #Расход газов
Tout = 80                                                                  #Температура газов на выходе

#Параметры охлаждающей среды:
fluidcond  = "REFPROP::WATER" #Охлаждающая среда
Tfluidcond = 15               #Температура охл. среды
Pfluidcond = 0.5              #Давление охл. среды

#Параметры ОЦР:
fluid   = "REFPROP::R236ea"   #Рабочая среда цикла
KPDpump = 0.8                 #КПД насоса
KPDturb = 0.85                 #КПД турбины
DTheat  = 10                  #Минимальный напор в нагревателе
DTcond  = 5                   #Минимальный напор в конденсаторе
Pcond   = prop.t_q(30,0,fluid)['P']               #Давление в конденсаторе

#Первое приближение

dTreg   = 5                #Минимальный напор в регенераторе
dPreg1  = 0                #Потери по горячему в МПа
dPreg2  = 0                #Потери по холодному


KPDold=0
Ppump   = 3.                 #Давление на выходе из насоса
for i in range(20):
    
    streams = pd.read_excel("streams.xlsx",index_col=0)
    blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

    #Внесение входных данных в таблицу:
    streams.loc["IN-HEAT", "T":"Q"] = [Tin,Pgas,prop.t_p(Tin,Pgas,gas)["H"],prop.t_p(Tin,Pgas,gas)["S"],Ggas,1]
    streams.loc["IN-COND", "T":"Q"] = [Tfluidcond,Pfluidcond,prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["H"],prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["S"],1000,0]

    #Начало расчёта - задание расхода на первой итерации:
    G = Ggas*(prop.t_p(Tin,Pgas,gas)["H"]-prop.t_p(Tout,Pgas,gas)["H"])/(prop.t_p(Tin,Ppump,fluid)["H"]-prop.t_p(Tout,Ppump,fluid)["H"])

    #Поток перед насосом:
    streams.loc["COND-PUMP", "T":"Q"] = [prop.p_q(Pcond,0,fluid)["T"],Pcond,prop.p_q(Pcond,0,fluid)['H'],prop.p_q(Pcond,0,fluid)['S'],G,0]

    #Итеративный расчет для сведения баланса:
    for i in range(100):
        #Расчет цикла:
        pump.calc     ("COND-PUMP", "PUMP-REG",                          Ppump,  KPDpump,     fluid, streams, blocks)
        regen.calc    ("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dTreg, dPreg1,dPreg2,fluid, streams, blocks)
        heater.calc   ("IN-HEAT",   "HEAT-OUT",  "REG-HEAT","HEAT-TURB", Tout,   DTheat, gas, fluid, streams, blocks)
        turbine.calc  ("HEAT-TURB", "TURB-REG",                          Pcond+dPreg1,  KPDturb,fluid, streams, blocks)
        regen.calc    ("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dTreg, dPreg1,dPreg2,fluid, streams, blocks)
        condenser.calc("REG-COND",  "COND-PUMP", "IN-COND" ,"COND-OUT",  DTcond, fluidcond,   fluid, streams, blocks)
        Qbalance = blocks.loc['HEATER','Q']+blocks.loc['PUMP','N']-blocks.loc['CONDENSER','Q']-blocks.loc['TURBINE','N']
        if abs(Qbalance) < 1: #Макс погрешность 1 кВт
            break

    KPD = (blocks.loc['TURBINE','N']-blocks.loc['PUMP','N'])/blocks.loc['HEATER','Q']
    
    if KPDold> KPD:
        break
        
    print(round(Ppump,2), round(KPD*100,5))
    Ppump = Ppump + 0.1
    KPDold = KPD

3.0 17.16552
3.1 17.24207
3.2 17.31256
3.3 17.37628
3.4 17.43367
3.5 17.4864
3.6 17.51086
3.7 17.52966
3.8 17.54553
3.9 17.55857
4.0 17.56965
4.1 17.57836
4.2 17.58524
4.3 17.58962
4.4 17.59213
4.5 17.59283


In [2]:
streams

,T,P,H,S,G,Q
IN-HEAT,183.600000,0.100000,615.516763,7.334542,509.000000,1.0
HEAT-OUT,80.000000,0.100000,503.727572,7.057047,509.000000,1.0
COND-PUMP,30.000000,0.244324,237.395788,1.129466,293.796164,0.0
PUMP-REG,32.275583,4.600000,241.238713,1.131985,293.807760,0.0
REG-HEAT,51.067851,4.600000,265.060688,1.207666,293.807760,0.0
HEAT-TURB,162.341184,4.600000,458.733274,1.709278,293.796164,1.0
TURB-REG,63.034202,0.244324,420.819848,1.729395,293.796164,1.0
REG-COND,37.275583,0.244324,396.996932,1.655683,293.796164,1.0
IN-COND,15.000000,0.500000,63.458441,0.224388,1000.270727,0.0
COND-OUT,26.207366,0.500000,110.335953,0.383989,1000.270727,0.0
